# Общие внутренние государственные расходы на здравоохранение на душу населения в долларах США

In [1]:
import pandas as pd

from helpers import get_data_with_full_locations_and_years

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/domestic-general-government-health-expenditure-(gghe-d)-per-capita-in-us

In [2]:
# Загрузим таблицу
data = pd.read_csv('../data/who_health_care_costs_per_capita.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,GHED_GGHE-D_pc_US_SHA2011,Domestic general government health expenditure...,numeric,EMR,Eastern Mediterranean,Country,SOM,Somalia,Year,2020,...,NaN,NaN,NaN,NaN,NaN,0.83,NaN,NaN,EN,2024-12-09T21:00:00.000Z
1,GHED_GGHE-D_pc_US_SHA2011,Domestic general government health expenditure...,numeric,AFR,Africa,Country,NGA,Nigeria,Year,2020,...,NaN,NaN,NaN,NaN,NaN,10.16,NaN,NaN,EN,2024-12-09T21:00:00.000Z


Описание нужных нам столбцов.

- `ParentLocationCode` - код региона
- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - общие внутренние государственные расходы на здравоохранение (GGHE-D) на душу населения в долларах США

Оставим только их.

In [3]:
data = data[[
    'ParentLocationCode', 'SpatialDimValueCode', 'Period', 'FactValueNumeric',
]]

data.rename(columns={'FactValueNumeric': 'HealthCareCostsPerCapita'}, inplace=True)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,HealthCareCostsPerCapita
0,EMR,SOM,2020,0.83
1,AFR,NGA,2020,10.16
2,AFR,UGA,2020,10.17
3,SEAR,MMR,2020,10.99
4,AMR,NIC,2020,102.60


В таблице для некоторых стран данные заполнены не за все года.\
Добавим строки с отсутствующими годами, чтобы явно увидеть пропуски.

In [4]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    }
)

# Посмотрим на пропуски
data.isna().sum()

ParentLocationCode           0
SpatialDimValueCode          0
Period                       0
HealthCareCostsPerCapita    77
dtype: int64

Заполним пропуски в пропущенные года средним по региону.

In [5]:
data['HealthCareCostsPerCapita'] = \
    data.groupby(['ParentLocationCode', 'Period'])['HealthCareCostsPerCapita'] \
    .transform(lambda x: x.fillna(x.mean()));

# Убедимся, что пропусков нет
data.isna().sum().sum()

0

In [6]:
# Удалим столбцы, которые больше не нужны
data.drop(columns=['ParentLocationCode'], inplace=True)

In [7]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_health_care_costs_per_capita_prepared.csv', index=False)